In [ ]:
import ifcopenshell
from topologicpy.Vertex import Vertex
from topologicpy.Face import Face
from topologicpy.Cluster import Cluster
from topologicpy.Topology import Topology
import re
import math

def load_ifc_file(file_path):
    """Load the IFC file."""
    return ifcopenshell.open(file_path)

def filter_ifcspaces_by_storey(spaces, storey_name):
    """Filter IfcSpaces by the given storey name."""
    filtered_spaces = []
    for space in spaces:
        for rel in space.Decomposes:
            if rel.is_a("IfcRelAggregates") and rel.RelatingObject.is_a("IfcBuildingStorey"):
                if rel.RelatingObject.Name == storey_name:
                    filtered_spaces.append(space)
                    break
    return filtered_spaces

def filter_spaces_by_name(spaces):
    """Filter IfcSpaces by name containing only digits (1-5 digits)."""
    digit_pattern = re.compile(r'^\d{1,5}$')
    filtered_spaces = []
    for space in spaces:
        if digit_pattern.match(space.Name):
            filtered_spaces.append(space)
    return filtered_spaces

def get_translation_vector(placement):
    """Extract the translation vector from the IfcAxis2Placement3D."""
    if placement and placement.Location:
        location = placement.Location
        return location.Coordinates[0], location.Coordinates[1], location.Coordinates[2]
    return 0.0, 0.0, 0.0

def get_direction_vector(ifc_file, direction):
    """Extract the direction vector from the IfcDirection."""
    if direction:
        dir_ref = ifc_file[direction.id()]
        if dir_ref:
            return tuple(dir_ref.DirectionRatios)  # Ensure the returned vector is a tuple
    return (0.0, 0.0, 1.0)  # Default direction

def get_axis_vector(placement):
    """Extract the axis vector from the IfcAxis2Placement3D."""
    if placement and placement.Axis:
        return tuple(placement.Axis.DirectionRatios)
    return (0.0, 0.0, 1.0)  # Default axis

def rotate_point(x, y, angle):
    """Rotate a point around the origin (0, 0) by a given angle in degrees."""
    radians = math.radians(angle)
    cos_angle = math.cos(radians)
    sin_angle = math.sin(radians)
    new_x = x * cos_angle - y * sin_angle
    new_y = x * sin_angle + y * cos_angle
    return new_x, new_y

def rotate_point_around_x(x, y, z, angle):
    """Rotate a point around the x-axis by a given angle in degrees."""
    radians = math.radians(angle)
    cos_angle = math.cos(radians)
    sin_angle = math.sin(radians)
    new_y = y * cos_angle - z * sin_angle
    new_z = y * sin_angle + z * cos_angle
    return x, new_y, new_z

def apply_transformation(coordinates, translation_vector, axis_vector, ref_direction_vector):
    """Apply translation and rotation to the coordinates."""
    transformed_coordinates = []

    for point in coordinates:
        x, y = point
        z = 0.0  # Since the original coordinates are 2D

        if axis_vector == (0.0, 0.0, -1.0):
            x, y, z = rotate_point_around_x(x, y, z, 180)  # Rotate 180 degrees around x-axis
            print(f"Axis Vector 0/0/-1 > Rotate 180 Degree around X-axis : Coordinates after Rotation: ({x}, {y}, {z})")

        if ref_direction_vector == (0.0, 1.0, 0.0):
            x, y = rotate_point(x, y, 90)  # Rotate 90 degrees
            print(f"Ref Direction Vector 0/1/0 > Rotate 90 Degree : Coordinates after Rotation: ({x}, {y})")
        elif ref_direction_vector == (-1.0, 0.0, 0.0):
            x, y = rotate_point(x, y, 180)  # Rotate 180 degrees
            print(f"Ref Direction Vector -1/0/0 > Rotate 180 Degree : Coordinates after Rotation: ({x}, {y})")
        elif ref_direction_vector == (0.0, -1.0, 0.0):
            x, y = rotate_point(x, y, 270)  # Rotate 270 degrees
            print(f"Ref Direction Vector 0/-1/0 > Rotate 270 Degree : Coordinates after Rotation: ({x}, {y})")
        
        # Apply translation
        new_x = x + translation_vector[0]
        new_y = y + translation_vector[1]
        new_z = z + translation_vector[2]
        transformed_coordinates.append((new_x, new_y, new_z))
        print(f"Coordinates after Translation: ({new_x}, {new_y}, {new_z})")

    return transformed_coordinates

def extract_face_from_space(space, ifc_file):
    """Extract face from the given space."""
    representation = space.Representation
    if representation:
        shape = representation.Representations[0]
        if shape and shape.Items:
            for item in shape.Items:
                if item.is_a("IfcExtrudedAreaSolid"):
                    profile = item.SweptArea
                    if profile and profile.is_a("IfcArbitraryClosedProfileDef"):
                        curve = profile.OuterCurve
                        if curve and curve.is_a("IfcIndexedPolyCurve"):
                            point_list = curve.Points
                            if point_list and point_list.is_a("IfcCartesianPointList2D"):
                                coordinates = point_list.CoordList
                                translation_vector = get_translation_vector(item.Position)
                                axis_vector = get_axis_vector(item.Position)
                                ref_direction_vector = get_direction_vector(ifc_file, item.Position.RefDirection)
                                if coordinates:
                                    print(f"Original Coordinates for {space.Name}: {coordinates}")
                                    print(f"Translation Vector: {translation_vector}")
                                    print(f"Axis Vector: {axis_vector}")
                                    print(f"Ref Direction Vector: {ref_direction_vector}")
                                    transformed_coordinates = apply_transformation(coordinates, translation_vector, axis_vector, ref_direction_vector)
                                    print(f"Total points: {len(transformed_coordinates)}")  # Total number of points
                                    print("-----")
                                    # Convert coordinates to Vertex objects
                                    vertices = [Vertex.ByCoordinates(x, y, z) for x, y, z in transformed_coordinates]
                                    # Create a Face from the vertices
                                    face = Face.ByVertices(vertices)
                                    return face
    return None

def main():
    # Load the IFC file
    ifc_file = load_ifc_file("Hus28_test.ifc")
    print("IFC file opened successfully.")
    
    # Define the storey name to filter
    storey_name = "Plan 10"
    
    # Find all IfcSpaces
    all_spaces = ifc_file.by_type("IfcSpace")
    num_spaces = len(all_spaces)
    print(f"Number of IfcSpaces found: {num_spaces}")
    
    # Filter spaces by the specified storey name
    spaces = filter_ifcspaces_by_storey(all_spaces, storey_name)
    num_filtered_spaces = len(spaces)
    print(f"Number of IfcSpaces matching storey '{storey_name}': {num_filtered_spaces}")
    
    # Further filter spaces by names containing only digits (1-5 digits)
    spaces = filter_spaces_by_name(spaces)
    num_filtered_by_name = len(spaces)
    print(f"Number of IfcSpaces with valid names: {num_filtered_by_name}")
    
    # List to hold all faces
    faces = []
    
    # Loop through each filtered IfcSpace and extract face
    for index, space in enumerate(spaces, start=1):
        print(f"Generating Face {index}/{num_filtered_by_name}")
        print(f"Name of Space: {space.Name}")
        face = extract_face_from_space(space, ifc_file)
        if face:
            faces.append(face)
        else:
            print(f"Could not generate face for space {index}.")
    
    # Create a Cluster from all faces
    if faces:
        building = Cluster.ByTopologies(faces)
        # Visualize the building using Topology
        Topology.Show(building)
    else:
        print("No faces generated.")

if __name__ == "__main__":
    main()